In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import decomposition
from matplotlib import pyplot as plt
from sklearn import neighbors

from double_CV_builder import DCV

In [2]:
# imports the data
data = pd.read_csv("../raw_data/train_call.txt", index_col=0, delimiter="\t").transpose()
topInfo = data.iloc[:3]
data = data.iloc[4:].dropna(axis=1)
classes = pd.read_csv("../raw_data/train_clinical.txt", delimiter="\t", index_col=0)
data = classes.join(data).set_index("Subgroup").dropna()
# Add 1 to data because of chi-square feature selection


DCV.outer_repeats = 5
DCV.inner_repeats = 10
DCV.num_features = 150

model = linear_model.LogisticRegression(penalty="elasticnet", max_iter=10000, multi_class='ovr', solver='saga')
modeller = DCV(model)
# prints all the params you can change :)
print(model.get_params().keys())
# select which params you want to test with the inner loop
# space is the paramater space
modeller.hyperParams['C'] = [100, 10, 1.0, 0.1, 0.01]
modeller.hyperParams['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]



dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])


In [3]:
# this does everything for you :)
# first is the data used, seconds comes the classifications and than with loop the amount of loops you want to do

modeller.train_fit(data=data, classes=data.index, loop=1)
# some test code

print(accuracy_score(DCV.__class_cleaner__(None, data.index),modeller.predict(data)))

Training progress: [--------------------------------------------------] 0.0% 
Training progress: [##########----------------------------------------] 20.0% 
Training progress: [####################------------------------------] 40.0% 
Training progress: [##############################--------------------] 60.0% 
Training progress: [########################################----------] 80.0% 
Training progress: [##################################################] 100.0% Done...

dict_values([0.1, 0.1])
[ 192  470  486  671  672  673  674  679  686  687  689  690  691  692
  693  694  695  696  697  698  699  700  701  702  818  837  839  840
  842  843  844  846  848  849  850  851  852  853  854  855  857  861
  998  999 1000 1001 1002 1003 1004 1008 1015 1016 1022 1025 1026 1027
 1028 1031 1032 1033 1034 1035 1036 1087 1089 1090 1091 1092 1563 1567
 1642 1643 1655 1656 1657 1663 1664 1670 1672 1673 1674 1675 1676 1677
 1678 1679 1895 1906 1907 1909 1910 1911 2004 2017 2019 2020 2021 20

In [5]:
print(topInfo.transpose().iloc[modeller.get_best_features()[0]])

                Start        End  Nclone
Chromosome                              
1           149369522  149394958       5
3           164038925  164101784       6
3           196908262  196937230       6
5            68785515   68850823       9
5            69274433   70345552       5
...               ...        ...     ...
17           77442683   77478901       2
17           77503378   77503685       3
17           77653668   77696106       6
22           22725750   22731544       2
22           37683612   37705310       5

[150 rows x 3 columns]


In [ ]:
modeller.all_accuracies

In [ ]:
modeller.all_accuracies.reset_index().pivot_table(index="l1_ratio",columns="C", aggfunc="mean")["Value"]

In [ ]:
cm = confusion_matrix(data.index, modeller.predict(data))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=modeller.get_best_model().classes_)
disp.plot()

In [ ]:
import scipy.stats as stats

stats.wilcoxon(modeller.predict(data), data.index)

In [ ]:
print(modeller.get_best_features())


#todo
box plots accuracy, precision, recall
best model. and params features

save all the juicy info (params, and features)

In [ ]:
print(model.__callable__("predict_proba")==True)